In [1]:
import sys
sys.path.append('../../..')
from hydradx.model.amm.omnipool_amm import OmnipoolState
from hydradx.model.amm.agents import Agent
from hydradx.model.processing import get_kraken_orderbook, get_omnipool_data, get_stableswap_data, get_orderbooks_from_file, get_binance_orderbook
from hydradx.model.processing import get_omnipool_data_from_file, get_centralized_market, load_stableswap_data, save_stableswap_data
from hydradx.model.amm.stableswap_amm import StableSwapPoolState 
from hydradx.model.amm.centralized_market import CentralizedMarket
import hydradx.model.amm.arbitrage_agent_general as arb

from pprint import pprint

import json

Set `get_live_markets = True` to get live markets from Kraken. Set to `False` to use order books from file.

In [2]:
get_live_markets = True
archive_data = False

In [3]:
# 2: Acala DAI
# 3: Acala WBTC
# 4: Acala WETH
# 5: DOT
# 9: ASTR
# 10: native USDT
# 11: iBTC
# 12: ZTG
# 13: CFG
# 14: BNC
# 15: vDOT
# 16: GLMR
# 17: INTR
# 18: Wormhole DAI
# 19: Wormhole WBTC
# 20: Wormhole WETH
# 100: 4-Pool

In [13]:
exchanges = {}
# load Omnipool
print('loading omnipool...')
RPC="wss://rpc.hydradx.cloud"
            
if get_live_markets:
    asset_list, asset_map, tokens, fees = get_omnipool_data(RPC, archive=archive_data)
else:
    asset_list, asset_map, tokens, fees = get_omnipool_data_from_file("archive/")
    
# pprint(tokens)
omnipool = OmnipoolState(
    tokens=tokens,
    lrna_fee={asset: fees[asset]['protocol_fee'] for asset in asset_list},
    asset_fee={asset: fees[asset]['asset_fee'] for asset in asset_list},
    preferred_stablecoin='USDT',
)
exchanges['omnipool'] = omnipool

loading omnipool...


In [14]:
# load stableswap pool
print('loading 4pool')

if get_live_markets:
    fourpool = get_stableswap_data()  # path='./archive' is default
    if archive_data:
        save_stableswap_data(fourpool)
else:
    fourpool = load_stableswap_data()
exchanges['4pool'] = fourpool

loading 4pool


In [15]:
# load centralized markets
print('loading centralized markets...')
# arb_file = "arbconfig.txt"
arb_files = ["arbconfig3-refactor.txt", "arbconfig-stableswap.txt"]

cfg = []
for arb_file in arb_files:
    with open('config/' + arb_file, 'r') as json_file:
        cfg += json.load(json_file)

for d in cfg:
    for ex in d['exchanges']:
        d['exchanges'][ex] = tuple(d['exchanges'][ex])

# pprint(cfg)
    
for exchange, trade_fee in {'kraken': 0.0016, 'binance': 0.0010}.items():
    if get_live_markets:
        exchanges[exchange] = get_centralized_market(
            config=cfg, 
            exchange_name=exchange, 
            trade_fee=trade_fee,
            archive=archive_data
        )
    else:
        exchanges[exchange] = CentralizedMarket(
            order_book=get_orderbooks_from_file("archive/")[exchange],
            unique_id=exchange,
            trade_fee=trade_fee
        )

loading centralized markets...


In [16]:
print('calculating arbitrage...')
all_swaps = arb.get_arb_swaps(exchanges, cfg)

calculating arbitrage...


In [17]:
test_exchanges = {ex_name: ex.copy() for ex_name, ex in exchanges.items()}
init_agent = Agent(
    holdings={tkn: 100000000 for tkn in [tkn for ex in exchanges.values() for tkn in ex.asset_list]}
)
test_agent = init_agent.copy()
arb.execute_arb(exchanges, test_agent, all_swaps)

equivalency_map = {
    'WETH': 'ETH',
    'XETH': 'ETH',
    'XXBT': 'BTC',
    'WBTC': 'BTC',
    'ZUSD': 'USD',
    'USDT': 'USD',
    'USDT10': 'USD',
    'USDT23': 'USD',
    'USDC': 'USD',
    'DAI': 'USD',
    'iBTC': 'BTC',
    'XBT': 'BTC',
}

profit = arb.calculate_profit(init_agent, test_agent, equivalency_map)

print("per-token profit")
print(profit)
print("total profit at Binance spot")
print(exchanges['binance'].value_assets(profit, equivalency_map))

# try rerouting to see if we can squeeze some more profit out of it
agent_2 = init_agent.copy()
test_ex_2 = {ex_name: ex.copy() for ex_name, ex in exchanges.items()}
all_swaps_2 = arb.combine_swaps(
    exchanges=test_ex_2,
    agent=agent_2,
    swaps=all_swaps,
    equivalency_map=equivalency_map
)
arb.execute_arb(test_ex_2, agent_2, all_swaps_2)
profit_2 = arb.calculate_profit(init_agent, agent_2, equivalency_map)

print("strategy 2 profit")
print(profit_2)
print("total at Binance spot")
print(exchanges['binance'].value_assets(profit_2, equivalency_map))


per-token profit
{'4-Pool': 0, 'HDX': 0, 'USD': 19.608704775571823, 'ETH': 0, 'GLMR': 0, 'BTC': 0.0, 'BNC': 0, 'vDOT': 0, 'CFG': 0.0, 'DOT': 0, 'PHA': 0, 'ZTG': 0, 'INTR': 0, 'ASTR': 0}
total profit at Binance spot
19.608704775571823
strategy 2 profit
{'4-Pool': 0, 'HDX': 0, 'USD': -9.716901570558548, 'ETH': 0, 'GLMR': 0, 'BTC': 3.68952751159668e-05, 'BNC': 0, 'vDOT': 0, 'CFG': 0.0, 'DOT': 0, 'PHA': 0, 'ZTG': 0, 'INTR': 0, 'ASTR': 0}
total at Binance spot
-8.126534183477593


In [ ]:
exchanges['4pool']


In [25]:
pprint(arb.flatten_swaps(all_swaps))
print('===================================')
pprint(all_swaps_2)

[{'amount': 0.078915013671875,
  'buy_asset': 'BTC',
  'exchange': 'binance',
  'price': 43126.442445,
  'sell_asset': 'USDT',
  'trade': 'buy'},
 {'amount': 0.078915013671875,
  'buy_asset': 'USDT',
  'exchange': 'omnipool',
  'min_buy': 3421.339453409553,
  'sell_asset': 'iBTC',
  'trade': 'sell'},
 {'amount': 396.068,
  'buy_asset': 'CFG',
  'exchange': 'omnipool',
  'max_sell': 249.07333614903686,
  'sell_asset': 'USDT',
  'trade': 'buy'},
 {'amount': 396.068,
  'buy_asset': 'USD',
  'exchange': 'kraken',
  'price': 0.6308844,
  'sell_asset': 'CFG',
  'trade': 'sell'},
 {'amount': 313.8125354960937,
  'buy_asset': 'CFG',
  'exchange': 'omnipool',
  'max_sell': 197.54348556518553,
  'sell_asset': 'USDT',
  'trade': 'buy'},
 {'amount': 313.8125354960937,
  'buy_asset': 'USD',
  'exchange': 'kraken',
  'price': 0.63078445,
  'sell_asset': 'CFG',
  'trade': 'sell'},
 {'amount': 0.009008288982391357,
  'buy_asset': 'BTC',
  'exchange': 'binance',
  'price': 43126.442445,
  'sell_asset':